In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os
import pathlib

p = pathlib.Path().resolve()

while True:
    if (p / ".git").exists():
        os.chdir(p)
        print("프로젝트 루트:", p)
        break

    if p.parent == p:
        raise RuntimeError("프로젝트 루트를 찾지 못함.")

    p = p.parent

프로젝트 루트: E:\AI_Project\codeit_ai_rfp_rag_assistant


In [6]:
from src.document_loader import get_hwp_text
test_file = "./Data/files/국민연금공단_2024년 이러닝시스템 운영 용역.hwp"

test = get_hwp_text(test_file)
print(test[:500])

2024년 이러닝시스템 운영 용역 제안요청서

2024년 이러닝시스템 운영 용역 제안요청서
 

청렴한 업무처리 및 부조리 신고
국민연금공단은 모든 계약업무를 투명하고 공정하게 처리하고 있습니다.
계약과정 및 계약이행 시 금품·향응·편의 제공을 요구하는 직원이 있거나, 지연·학연 등의 사유로 특혜를 제공하는 것을 보거나 들은 경우에는 공단 홈페이지 부조리신고센터로 신고하여 주시기 바랍니다.


부조리
신고센터
국민연금 홈페이지(www.nps.or.kr)  클린센터

레드휘슬(www.redwhistle.org)익명성 보장
2024. 1.

인사혁신실 인재개발부


목   차


 . 사업 안내	   	1
   1. 사업개요	   	1
   2. 사업목적	   	1
   3. 사업범위	   	1
   4. 기대효과	   	2
 . 제안요청 내용	   	3
   1. 사업 운영방향	   	3
  


In [10]:
import src.vectorstore as vectorstore
import src.chunking as chunking

file_path = './Data/files'
chunk = chunking.load_and_chunk_all_docs(file_path)
vector_store = vectorstore.build_vector_store(chunk)

retriever = vector_store.as_retriever(search_kwargs={'k': 3})

✅ (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp 청킹 완료 (84개)
✅ (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.hwp 청킹 완료 (43개)
✅ (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.hwp 청킹 완료 (60개)
✅ (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.hwp 청킹 완료 (35개)
✅ 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.hwp 청킹 완료 (39개)
✅ BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).hwp 청킹 완료 (57개)
✅ KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .hwp 청킹 완료 (107개)
✅ 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.hwp 청킹 완료 (53개)
✅ 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.hwp 청킹 완료 (68개)
✅ 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.hwp 청킹 완료 (58개)
✅ 경상북도 봉화군_봉화군 재난통합관리시

In [11]:
for i, doc in enumerate(chunk[:3]):
    print(f"\n[{i+1}] 문서명: {doc.metadata.get('source', '미상')}")
    print(doc.metadata)


[1] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp', 'org': '(사)벤처기업협회', 'category': '2024년 벤처확인종합관리시스템 기능 고도화 용역사업 입찰공고', 'doc_type': '제안요청서', 'budget': '352000000.0', 'open_date': '2024-03-19 10:49:46', 'start_date': 'nan', 'end_date': '2024-04-09 15:00:00'}

[2] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp', 'org': '(사)벤처기업협회', 'category': '2024년 벤처확인종합관리시스템 기능 고도화 용역사업 입찰공고', 'doc_type': '제안요청서', 'budget': '352000000.0', 'open_date': '2024-03-19 10:49:46', 'start_date': 'nan', 'end_date': '2024-04-09 15:00:00'}

[3] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용

In [12]:
docs = vector_store.get(include=["metadatas", "documents"])
print(len(docs["documents"]))

12192


In [13]:
import unicodedata

target_keyword = "이러닝"  # 찾고 싶은 키워드

docs = vector_store.get(include=["metadatas", "documents"], limit=99999)
found = 0

for i, (meta, content) in enumerate(zip(docs["metadatas"], docs["documents"])):
    src = unicodedata.normalize("NFC", str(meta.get("source", ""))).strip()

    # '이러닝' 또는 자모분리형 '이러닝' 모두 검색
    if target_keyword in src or "이러닝" in src:
        print(f"🧾 {i}번 문서명: {src}")
        print(f"본문 길이: {len(content)}자")
        print(f"앞 200자 미리보기:\n{content[:200]}")
        print("="*80)
        found += 1
        break

🧾 1365번 문서명: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp
본문 길이: 1484자
앞 200자 미리보기:
2024년 이러닝시스템 운영 용역 제안요청서

2024년 이러닝시스템 운영 용역 제안요청서
 

청렴한 업무처리 및 부조리 신고
국민연금공단은 모든 계약업무를 투명하고 공정하게 처리하고 있습니다.
계약과정 및 계약이행 시 금품·향응·편의 제공을 요구하는 직원이 있거나, 지연·학연 등의 사유로 특혜를 제공하는 것을 보거나 들은 경우에는


In [14]:
from src.rag_chain import run_chain

result = run_chain(vector_store, retriever, "국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항 정리해줘")
print(result)

📄 자동선택된 문서: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (유사도 0.476)
📄 자동선택된 문서: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (유사도 0.476)
📄 자동선택된 문서: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (유사도 0.476)
📄 자동선택된 문서: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (유사도 0.476)
📄 자동선택된 문서: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (유사도 0.476)
📄 자동선택된 문서: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (유사도 0.476)
📄 자동선택된 문서: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (유사도 0.476)
핵심 요약
- 국민연금공단이 발주한 2024년 이러닝시스템 운영용역은 비대면 교육 수요 대응과 직원 역량 강화를 목적으로 이러닝(LMS) 운영, 직무·부서별 콘텐츠 개발·관리, 외부 위탁(자기개발) 과정 운영을 포함한다(공단 문서 근거).  
- 계약기간은 체결일부터 2025년 2월까지이며, 전체 예산은 773,801,000원이고 예산은 교육과정 운영(공단 소유·외부콘텐츠)과 콘텐츠·시스템(개발·관리·서버임대)으로 구분되어 있다(공단 문서 근거).

세부 설명 — 요구사항 구성(문서 근거에 따라 정리)
1) 사업 목적(문서 근거)
- 비대면(온택트) 교육 수요 충족.  
- 직급별 역량진단을 통한 맞춤형 교육 제공(상반기: 사전 역량진단, 하반기: 사후 역량진단, 진단 문항은 공단 제공).  
- 직무·부서별 문제해결 역량 향상 목적의 콘텐츠 개발.  
- 이러닝 시스템 운영을 통한 상시 학습 활성화·효율화.

2) 사업 범위(문서 근거)
- 직무 콘텐츠 개발·운영.  
- 위탁교육(자기개발) 콘텐츠 운영.  
(문서 목차에 따르면 요구사항은 2-1 교육과정 운영, 2-2 이러닝시스템 운영, 2-3 콘텐츠 개발·관리, 2-4 개인정보보호 및 정보보안, 2-5 기타사항으로 구성되어 있음)

3) 예

In [15]:
result = run_chain(vector_store, retriever, "한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘")
print(result)

📄 자동선택된 문서: 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (유사도 0.430)
📄 자동선택된 문서: 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (유사도 0.430)
📄 자동선택된 문서: 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (유사도 0.430)📄 자동선택된 문서: 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (유사도 0.430)

📄 자동선택된 문서: 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (유사도 0.430)
📄 자동선택된 문서: 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (유사도 0.430)
📄 자동선택된 문서: 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (유사도 0.430)
핵심 요약
- 문서에 따르면 본 사업은 원자력안전 규제요건을 충족하고(연간선량 준수 확인체계 구축), 기존 선량평가 방법·시스템의 정확성과 기능을 개선(평가장기 개선·액체 유출물 평가 추가)하며 UI·출력 고도화를 통해 업무생산성을 높이기 위해 추진됩니다.
- 기대효과로는 정상운전 시 규제 수요 대응, 데이터 신뢰성 확보 및 생산성 향상이 명시되어 있습니다.

세부 설명 (문서 근거 중심)
1) 규제요건 준수 필요
- 원자력안전위원회 고시(제2019-10호)에서 요구하는 제한구역 경계의 연간선량 준수 여부를 확인할 수 있는 체계 구축 필요성이 제기되어 사업을 추진합니다(문서: 추진배경 및 필요성).

2) 평가방법·모듈 개선 필요
- 기존 ICRP60 기반 평가장기(organ)에 대한 개선이 필요하다고 명시되어 있어, 평가장기 모듈(포트란 및 웹시스템)을 개선하는 작업이 포함됩니다(문서: 평가장기 개선).

3) 액체 유출물에 의한 선량평가 기능 추가
- 액체 유출물로 인한 주민 피폭선량 평가 수행이 필요하다는 점이 명시되어 있어, 포트란 모듈 및 웹시스템에 액체 선량평가 기능을 추가하는 것이 사업 목적에 포함됩니다(문서: 액체유출물에 의한 주민피

In [16]:
result = run_chain(vector_store, retriever, "기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?")
print(result)

📄 자동선택된 문서: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf (유사도 0.381)
📄 자동선택된 문서: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf (유사도 0.381)
📄 자동선택된 문서: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf (유사도 0.381)
📄 자동선택된 문서: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf (유사도 0.381)
📄 자동선택된 문서: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf (유사도 0.381)
📄 자동선택된 문서: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf (유사도 0.381)
📄 자동선택된 문서: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf (유사도 0.381)
핵심 요약
- 과업지시서(2024.10.)를 확인한 결과, AI 기반 예측(예: AI, 머신러닝, 예측모델 등)에 대한 별도 요구사항이나 명시적 언급은 없습니다.  
- 따라서 해당 사업(2025년도 중이온가속기용 극저온시스템 운전 용역) 문서상에는 AI 예측 도입을 의무화하거나 요구하는 항목이 존재하지 않습니다.

세부 설명
- 기초과학연구원 중이온가속기연구소가작성한 "2025년도 중이온가속기용 극저온시스템운전 용역 과업지시서(2024.10.)"를 근거로 확인한 결과, 문서 내에 “AI”, “머신러닝”, “예측”, “예측모델” 등의 용어와 이에 따른 요구사항이나 산출물 규정은 기재되어 있지 않습니다.  
- 문서에서 요구하는 작업 범위나 산출물 중 AI 기반 예측 기능을 명시한 항목이 없으므로, 현재 문서만으로는 AI 예측 관련 의무 요건이 있다고 볼 근거가 없습니다.

관련 수치/일정
- 사업명: 2025년도 중이온가속기용 극저온시스템 운전 용역  
- 예산: 743,070,000원  
- 공고(공개)일: 2024-11-15, 입찰마감: 2024-11-26 14:00



In [17]:
result = run_chain(vector_store, retriever, "고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?")
print(result)

📄 자동선택된 문서: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (유사도 0.735)
📄 자동선택된 문서: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (유사도 0.735)
📄 자동선택된 문서: 광주과학기술원_학사시스템 기능개선 사업.hwp (유사도 0.833)
📄 자동선택된 문서: 광주과학기술원_학사시스템 기능개선 사업.hwp (유사도 0.833)
📄 자동선택된 문서: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (유사도 0.735)
📄 자동선택된 문서: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (유사도 0.735)
📄 자동선택된 문서: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (유사도 0.735)
📄 자동선택된 문서: 광주과학기술원_학사시스템 기능개선 사업.hwp (유사도 0.833)
📄 자동선택된 문서: 광주과학기술원_학사시스템 기능개선 사업.hwp (유사도 0.833)
📄 자동선택된 문서: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (유사도 0.735)
📄 자동선택된 문서: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (유사도 0.735)
📄 자동선택된 문서: 광주과학기술원_학사시스템 기능개선 사업.hwp (유사도 0.833)
📄 자동선택된 문서: 광주과학기술원_학사시스템 기능개선 사업.hwp (유사도 0.833)
📄 자동선택된 문서: 광주과학기술원_학사시스템 기능개선 사업.hwp (유사도 0.833)
핵심 요약
- 두 사업은 발주기관·규모·목적에서 뚜렷히 다릅니다. 고려대학교 사업은 대규모의 "차세대 포털·학사 정보시스템 구축"(예산 11,270,000,000원)이고, 광주과학기술원(GIST) 사업은 상대적으로 소규모의 "학사시스템 기능개선"(예산 157,300,000원)입니다.  
- 문서상 확인되는 공개일·입찰마감일과 GIST의 계약기간(계약체결일로부터 5개월)이 다릅니다.

세부 설명

1) 공통점 (문서 근거 기준)
- 둘 다 대학(고려대학교 / 광주과학기술원

In [18]:
result = run_chain(vector_store, retriever, "벤처기업협회의 벤처확인종합관리시스템 기능 고도화 용역사업에서 추진일정을 주요내용별로 정리해줘")
print(result)

📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.653)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.653)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.653)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.653)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.653)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.653)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.653)
핵심 요약
- 본 문서에 따르면 사업 전체 기간은 '계약일로부터 150일'이며, 주요 마일스톤으로 1차 오픈(2024.07.10, 성과조건부 및 스톡옵션)과 2차 오픈(2024년 9월, GrandOpen)이 명시되어 있습니다.  
- 개별 기능별(복수의결권주식, 스톡옵션 이관·신규구축, 성과조건부주식) 세부 추진일정은 문서에 구체적 일자·기간으로는 명시되어 있지 않습니다(언급 없음).

세부 설명 — 주요 내용별 추진일정(문서 근거 기준)
1) 사업기간 및 전체 마일스톤
- 사업기간: 계약일로부터 150일(문서 명시).  
- 1차 오픈: 2024.07.10 — 문서에는 1차 오픈 시점에 "성과조건부 및 스톡옵션 오픈"이라고 명시.  
- 2차 오픈: 2024년 9월(월 단위 표기) — 문서 표기: "GrandOpen".  
- 예산: 352,000,000원(부가세 포함) — 참고 수치.

2) 복수의결권주식 관련
- 추진내용(문서): 벤처기업법 제16조의11에 따른 복수의결권주식 발행 보고 업무처리 시스템 구축

In [19]:
result = run_chain(vector_store, retriever, "벤처기업협회의 벤처확인종합관리시스템 기능 고도화 용역사업에서 소프트웨어 개발사업의 적정 사업기간 종합 산정서의 검토의견 정리해줘")
print(result)

📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.539)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.539)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.539)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.539)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.539)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.539)
📄 자동선택된 문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp (유사도 0.539)
핵심 요약
- 문서상 본 사업의 전체 사업기간은 "계약일로부터 150일"로 규정되어 있고, 산출된 주요 마일스톤은 1차 오픈(2024.07.10) 및 2차 GrandOpen(2024.09월)으로 명시되어 있음(문서 근거).  
- 그러나 계약일·세부 일정 분해(WBS)·자원·테스트·데이터 이관 규모 등 사업기간 적정성을 판단할 핵심 근거가 문서에 제시되어 있지 않아, 현재 제출된 정보만으로는 150일이 적정한지 판정할 수 없음(언급 없음).  

세부 설명 (논리적 흐름)
1) 문서에 명시된 일정 근거
   - 사업기간: 계약일로부터 150일 (문서)  
   - 마일스톤: 1차 오픈 2024.07.10 (성과조건부·스톡옵션 오픈), 2차 오픈 2024.09월 GrandOpen (문서)  
   - 사업범위: 복수의결권주식·스톡옵션·성과조건부주식 기능 구축 및 고도화, 과거 데이터 정제·이관 포함(문서)  
   - 예산: 352,000,000원(부가세 포함)(문서)

2) 적정성 판단을 위한 현재 문서상의 정보